In [1]:
# Import Statements

# Sleeper wrapper
from sleeper_wrapper import League

# Progress Bar
from tqdm import tqdm

# Misc
import requests, pandas as pd

In [2]:
# Find your league
league = League(728718179756519424)

In [3]:
# Query sleeper API for list of all players
result_json_string = requests.get("https://api.sleeper.app/v1/players/nfl");

# Try to query
try:
    result_json_string.raise_for_status()
except requests.exceptions.HTTPError as e:
    print(e)

# Convert to json
players_json = result_json_string.json()

In [4]:
# Function to parse the players json and turn it into a dataframe
def parse_players(players_json):
    # Create lists for data that we care about
    last_names = []
    first_names = []
    positions = []
    ids = []

    # Loop through the dictionary
    for k,v in tqdm(players_json.items()):
        # Create a temp dict
        item = players_json[k]
        # Save the metadata we carry about
        last_names.append(item['last_name'])
        first_names.append(item['first_name'])
        positions.append(item['position'])
        ids.append(item['player_id'])
    
    # Create a list of full/whole names
    names = ["{} {}".format(x,y) for x,y in zip(first_names, last_names)]

    # Convert to dataframe
    res_df = pd.DataFrame({'Name': names, 'First Name': first_names, 'Last Name': last_names, 'Position': positions, 'ID': ids})
    
    # Return it
    return res_df

In [5]:
# Get the players dataframe
players_df = parse_players(players_json)

100%|██████████| 8094/8094 [00:00<00:00, 867964.53it/s]


In [8]:
t = league.get_matchups(10)

In [7]:
lookup_dict = {1: 'Robby', 2: 'Matt', 3: 'Tony', 4: 'Tim', 5: 'Kade', 6: 'BJ', 7: 'Erik', 8: 'Joey', 9: 'Ric', 10: 'Chris'}

In [9]:
test_df = pd.json_normalize(t).explode(['starters', 'starters_points'])
test_df = test_df[test_df.columns.drop(list(test_df.filter(regex="players_points.")))]
test_df.drop('custom_points', inplace=True, axis=1)
test_df['roster_id'] = test_df['roster_id'].map(lookup_dict)

In [10]:
test_df

,starters_points,starters,roster_id,points,players,matchup_id
0,26.34,3294,Robby,110.6,"[TEN, NE, 7606, 7564, 7561, 6783, 6770, 6083, ...",5
0,9.1,7561,Robby,110.6,"[TEN, NE, 7606, 7564, 7561, 6783, 6770, 6083, ...",5
0,18.26,3164,Robby,110.6,"[TEN, NE, 7606, 7564, 7561, 6783, 6770, 6083, ...",5
0,8.8,5937,Robby,110.6,"[TEN, NE, 7606, 7564, 7561, 6783, 6770, 6083, ...",5
0,2.5,1144,Robby,110.6,"[TEN, NE, 7606, 7564, 7561, 6783, 6770, 6083, ...",5
...,...,...,...,...,...,...
9,18.3,5012,Chris,106.5,"[CLE, 6813, 6786, 6151, 5980, 5955, 5892, 5849...",4
9,4.8,5022,Chris,106.5,"[CLE, 6813, 6786, 6151, 5980, 5955, 5892, 5849...",4
9,0.0,5110,Chris,106.5,"[CLE, 6813, 6786, 6151, 5980, 5955, 5892, 5849...",4
9,12.0,1945,Chris,106.5,"[CLE, 6813, 6786, 6151, 5980, 5955, 5892, 5849...",4


In [18]:
def get_team_data(df):
    winners = []
    losers = []
    scores = {}
    games = {}

    for matchup in df['matchup_id'].unique():        
        
        # Limit the data to just the current matchup
        matchup_df = df.loc[df['matchup_id']==matchup]

        print("-----------")
        print(matchup_df)
        print("-----------")

        a = [matchup_df['roster_id'].unique()[0], matchup_df['points'].unique(), matchup_df['roster_id'].unique()[0], matchup_df['points'].unique()]
        b = [matchup_df['roster_id'].unique()[1], matchup_df['points'].unique(), matchup_df['roster_id'].unique()[1], matchup_df['points'].unique()]
        
        print(a, b)

        # print(matchup_df)
        # total_points_df = matchup_df.groupby('roster_id').sum('starters_points').reset_index()
        # total_points_df = total_points_df.round(3)
        # games[matchup] = total_points_df['starters_points'].sum().round(3)
        # scores[total_points_df['roster_id'][0]] = total_points_df['starters_points'][0]
        # scores[total_points_df['roster_id'][1]] = total_points_df['starters_points'][1]

        # if total_points_df['starters_points'][0] > total_points_df['starters_points'][1]:
        #     w = "{} (vs {})".format(total_points_df['roster_id'][0], total_points_df['roster_id'][1])
        #     l = "{} (vs {})".format(total_points_df['roster_id'][1], total_points_df['roster_id'][0])
        #     winners.append(w)
        #     losers.append(l)
        # elif total_points_df['starters_points'][0] < total_points_df['starters_points'][1]:
        #     w = "{} (vs {})".format(total_points_df['roster_id'][1], total_points_df['roster_id'][0])
        #     l = "{} (vs {})".format(total_points_df['roster_id'][0], total_points_df['roster_id'][1])
        #     winners.append(w)
        #     losers.append(l)
        # else:
        #     w = "{} tied {}!".format(total_points_df['roster_id'][0], total_points_df['roster_id'][1])
        #     l = w
        #     winners.append(w)
        #     losers.append(l)
    
    # top_scorer = max(scores.items(), key = lambda k : k[1])
    # low_game = list(min(games.items(), key = lambda k : k[1]))
    # top_game = list(max(games.items(), key = lambda k : k[1]))
    # top_game[0] = list(matchup_df.loc[matchup_df['matchup_id']==top_game[0]]['roster_id'].unique())
    # low_game[0] = list(matchup_df.loc[matchup_df['matchup_id']==low_game[0]]['roster_id'].unique())
    # return "WINNERS: {}".format(', '.join(winners)), "LOSERS: {}".format(', '.join(losers)), list(top_scorer), list(top_game), list(low_game)

In [19]:
get_team_data(test_df)

# winners, losers, scores, high, low = get_team_data(test_df)
# print(winners)
# print(losers)
# print(scores)
# print(high)
# print(low)

-----------
  starters_points starters roster_id  points  \
0           26.34     3294     Robby  110.60   
0             9.1     7561     Robby  110.60   
0           18.26     3164     Robby  110.60   
0             8.8     5937     Robby  110.60   
0             2.5     1144     Robby  110.60   
0            19.9     1466     Robby  110.60   
0             8.6     1339     Robby  110.60   
0             7.1     6783     Robby  110.60   
0             4.0     6083     Robby  110.60   
0             6.0      TEN     Robby  110.60   
3           11.82      421       Tim   81.12   
3             3.9     5347       Tim   81.12   
3            10.6     4199       Tim   81.12   
3             2.8     5859       Tim   81.12   
3             3.9     5045       Tim   81.12   
3             0.0     4993       Tim   81.12   
3             8.9     5927       Tim   81.12   
3             9.2     4037       Tim   81.12   
3             6.0      391       Tim   81.12   
3            24.0      DAL  

TypeError: 'numpy.float64' object is not iterable

In [82]:
games

(3, 2429.6)

In [31]:
users = league.get_rosters()